# 1) Import Libraries

In [332]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# 2) Read Dataset

In [270]:
# books = pd.read_csv("dataset - amazon - books.csv",sep=";",error_bad_lines=False, encoding="latin-1", low_memory=False)
# ratings = pd.read_csv("dataset - amazon - ratings.csv",sep=";",error_bad_lines=False, encoding="latin-1", low_memory=False)
# users = pd.read_csv("dataset - amazon - users.csv",sep=";",error_bad_lines=False, encoding="latin-1", low_memory=False)

countries_combine_pivot = pd.read_csv('implementation - presentation - booksPivot.csv', index_col=0)
books3 = pd.read_csv('implementation - presentation - books727.csv', index_col=0)

# ONE TIME RUN

## Users with less than 50 ratings and books less than 100 ratings are excluded. (ONE TIME RUN)

In [262]:
# Users with less than 50 ratings are excluded:
counts1 = ratings['User-ID'].value_counts()
ratings1 = ratings[ratings['User-ID'].isin(counts1[counts1 >= 50].index)]
# Books less than 100 ratings are excluded:
counts2 = ratings['ISBN'].value_counts()
ratings2 = ratings[ratings['ISBN'].isin(counts2[counts2 >= 100].index)]

## Collaborative Filtering Using k-Nearest Neighbors(KNN) (ONE TIME RUN)

In [263]:
# books1:
books1 = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]

# Merge books1 & ratings2
combine_book_rating = pd.merge(ratings2, books1, on='ISBN')
combine_books = combine_book_rating.drop(["Book-Author","Year-Of-Publication","Publisher","Image-URL-L"],axis = 1)

# Added Total_Rating_Count Column
book_ratings_count = (combine_books.groupby(by = ['Book-Title'])['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating' :'Total_Rating_Count'})[['Book-Title','Total_Rating_Count']])
book_ratings_count = book_ratings_count.sort_values(by = 'Total_Rating_Count',ascending=False)

# Merge Total_Rating_Count Column df with Merge of books1 & ratings2 (combine_books)
combine_books1 = combine_books.merge(book_ratings_count,on = 'Book-Title')

# Exclude books having Total_Rating_Count less than 30:
popularity_threshold = 30
combine_books2 = combine_books1.query('Total_Rating_Count >= @popularity_threshold')

# Merge combine_books2 with users df
combine_books3 = combine_books2.merge(users,on='User-ID')

# Include books of only 7 main countries:
countries_combine = combine_books3[combine_books3['Location'].str.contains('usa|canada|united kingdom|germany|spain|australia|italy')]
countries_combine = countries_combine.drop(['Age','Location'],axis = 1)


# Creating Matrix for KNN
countries_combine = countries_combine.drop_duplicates(['User-ID', 'Book-Title'])

# countries_combine_pivot = Book-Title vs User-ID Table
countries_combine_pivot = countries_combine.pivot(index= 'Book-Title',columns = 'User-ID',values = 'Book-Rating').fillna(0)

# Uses Sparse Matrix
countries_combine_matrix = csr_matrix(countries_combine_pivot.values)

# KNN Algorithm
knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

# Fit Matrix into KNN Model
knn_model.fit(countries_combine_matrix)

# Export countries_combine_pivot
countries_combine_pivot.to_csv('implementation - presentation - booksPivot.csv', index=True)

## ADD Avg-Rating Column (ONE TIME RUN)

In [264]:
# Find Averages of Books:

# combine_books = [User-ID, ISBN, Book-Rating, Book-Title]
# combine_books11 = exclude all the books which have 0 ratings in total
combine_books11 = combine_books[combine_books['Book-Rating']>=1]
# title_list = list of Book-Title of combine_books11
title_list = list(set(combine_books11['Book-Title'].tolist()))

# avg_rating = [Book-Title, Avg rating]
avg_rating = []
# Find avg book rating of all books and rounding the avg to 1 decimal:
for i in title_list:
    avg_rating.append([i,round(combine_books11[combine_books11['Book-Title']==i]['Book-Rating'].mean(),1)])
    

    
# Remove NA values from books1
books2 = books1.dropna()
books2.reset_index(inplace = True, drop = True)


# Add Avg-Rating column to books2
for i in avg_rating:
#     index_list = list of indices which match the Book-Title in books2
    index_list = list(books2[books2['Book-Title']==i[0]].index)
#     Add value to Avg-Rating column of books2 using index_list
    books2.loc[index_list, 'Avg-Rating'] = i[1]
    
    
# books3 = 1915 filtered books
books3 = books2.dropna()
# removed duplicate ISBN books from countries_combine
countries_combine_ISBN_list = list(set(countries_combine.ISBN))
books3 = books3[books3['ISBN'].isin(countries_combine_ISBN_list)]
books3.reset_index(inplace = True, drop = True)
books3.to_csv('implementation - presentation - books727.csv')

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Find Random Book

In [222]:
# Index of Book-Title from Book-Title vs User-ID Pivot Table
query_index = np.random.choice(countries_combine_pivot.shape[0])
# Index of Book-Title from Book-Title vs User-ID Pivot Table
query_name = countries_combine_pivot.index[query_index]

print(query_index)
print(query_name)

531
The Last Juror


# KNN (RUN Everytime)

# Final Output

In [384]:
def runKNN(countries_combine_pivot):
    # Uses Sparse Matrix
    countries_combine_matrix = csr_matrix(countries_combine_pivot.values)
    # KNN Algorithm
    knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    # Fit Matrix into KNN Model
    knn_model.fit(countries_combine_matrix)
    return knn_model


# RECOMMENDED BOOKS LIST FOR ONE BOOK
def recommendCollabOneBook(books3, countries_combine_pivot, ISBN, user_recommendation):
    bookName = list(books3[books3['ISBN']==ISBN]['Book-Title'])[0]
    query_index = list(countries_combine_pivot.index).index(bookName)
    # countries_combine_pivot = Book-Title vs User-ID Table
    # .iloc -> Display row of query_index with User-ID
    # .values -> convert the .iloc to values
    # .reshape(1,-1) -> Convert the single array into array of array
    # .reshape(1,-1) -> .reshape(rows, columns) {1 value indicates 1 row, -1 indicates no proper value for columns}
    countries_combine_reshape = countries_combine_pivot.iloc[query_index,:].values.reshape(1,-1)
    # run KNN Model
    knn_model = runKNN(countries_combine_pivot)
    # distances and indices of output:
    distances, indices = knn_model.kneighbors(countries_combine_reshape, n_neighbors = user_recommendation+1)
    # Final names list for one book
    recommend_collab_names = []
    for i in indices[0][1:]:
        recommend_collab_names.append(countries_combine_pivot.index[i])
    return recommend_collab_names


# FINAL RECOMMENDED COMBINED BOOKS LIST
def recommendCollab(books3, countries_combine_pivot, books_list, user_recommendations):
    # Filter 3+ ratings books    
    high_rated_list = []
    for i in books_list:
        if i[1]>=3:
            high_rated_list.append(i[0])
    # Combine recommended books list    
    recommend_collab_names = []
    for i in high_rated_list:
        recommend_collab_names = recommend_collab_names + recommendCollabOneBook(books3, countries_combine_pivot, i, user_recommendations)
    # Final 10 recommended books list
    # recommend_collab_names = random.sample(recommend_collab_names, user_recommendations)
    return recommend_collab_names


def getBooksDataframeFromNames(books3, recommend_collab_names):
    recommend_collab = books3[books3['Book-Title'].isin(recommend_collab_names)].groupby('Book-Title').first()
    recommend_collab = recommend_collab.sort_values(by='Avg-Rating', ascending=False)
    recommend_collab.reset_index(inplace=True, drop=True)
    return recommend_collab.head(10)

In [389]:
user_book_ISBN = list(books3.sample(5).ISBN.values)
user_book_rating = [5,1,1,5,4]
books_list = list(map(lambda x, y: [x,y], user_book_ISBN, user_book_rating))
user_recommendations = 10

recommend_collab_names = recommendCollab(books3, countries_combine_pivot, books_list, user_recommendations)
recommend_collab = getBooksDataframeFromNames(books3, recommend_collab_names)

In [390]:
recommend_collab

,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Avg-Rating
0,0060987561,Wally Lamb,1999,Regan Books,http://images.amazon.com/images/P/0060987561.0...,8.3
1,0553273914,Isabel Allende,1986,Bantam Books,http://images.amazon.com/images/P/0553273914.0...,8.2
2,0446672211,Billie Letts,1998,Warner Books,http://images.amazon.com/images/P/0446672211.0...,8.1
3,0312980140,Janet Evanovich,2002,St. Martin's Paperbacks,http://images.amazon.com/images/P/0312980140.0...,8.1
4,0804105820,Robert Fulghum,1993,Ivy Books,http://images.amazon.com/images/P/0804105820.0...,8.1
5,0373484003,Nora Roberts,2000,Silhouette,http://images.amazon.com/images/P/0373484003.0...,8.0
6,0515121843,Nora Roberts,2001,Jove Books,http://images.amazon.com/images/P/0515121843.0...,7.9
7,0375706771,Chris Bohjalian,1998,Vintage,http://images.amazon.com/images/P/0375706771.0...,7.7
8,0671021001,Wally Lamb,1998,Pocket,http://images.amazon.com/images/P/0671021001.0...,7.7
9,0679735909,A.S. BYATT,1991,Vintage,http://images.amazon.com/images/P/0679735909.0...,7.7


# 1

In [57]:
def getImageURL(input1):
    return books2[books2['ISBN']==input1]['Image-URL-L']

In [58]:
getImageURL()

TypeError: getImageURL() missing 1 required positional argument: 'input1'